In [1]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, NonNegativeReals, Suffix
from pyomo.opt import SolverFactory
import pandas as pd

In [2]:
gens = ["gas","coal"]

data = pd.DataFrame(index=gens)

data["costs"] = [20,8]
data["capacities"] = [350,300]
data["efficiencies"] = [0.6,0.4]
data["emissions"] = [0.2,0.3]

data

,costs,capacities,efficiencies,emissions
gas,20,350,0.6,0.2
coal,8,300,0.4,0.3


In [12]:
data["costs_el"] = data.costs/data.efficiencies
data.costs_el

gas     33.333333
coal    20.000000
Name: costs_el, dtype: float64

In [13]:
data["emissions_el"] = data.emissions/data.efficiencies
data.emissions_el

gas     0.333333
coal    0.750000
Name: emissions_el, dtype: float64

In [23]:
demand = 500
co2_limit = 250
#co2_limit = 300

In [24]:
model = ConcreteModel()

model.x = Var(gens, domain=NonNegativeReals)

model.balance = Constraint(expr = model.x["gas"] + model.x["coal"] == demand)

def cap_limits(model,gen):
    return model.x[gen] <= data.capacities[gen]

model.cap_limits = Constraint(gens,rule=cap_limits)

model.emissions = Constraint(expr = sum([data.emissions_el[gen]*model.x[gen] for gen in gens]) <= co2_limit)

model.objective = Objective(expr = sum([data.costs_el[gen]*model.x[gen] for gen in gens]))

In [25]:
opt = SolverFactory("glpk")

In [26]:
model.dual = Suffix(direction=Suffix.IMPORT_EXPORT)

In [27]:
results = opt.solve(model,suffixes=["dual"])

In [28]:
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 14000.0
  Upper bound: 14000.0
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 3
  Number of nonzeros: 7
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.010906696319580078
# ----------------------------------------------------------
#   Solution Information
# --------------------------------

In [29]:
for gen in gens:
    print(gen,model.x[gen].value)

gas 300.0
coal 200.0


In [30]:
model.dual[model.balance]

44.0

In [31]:
model.dual[model.emissions]

-32.0